# Homework 4

**Due 10/6/2020 on gradescope.**

## References

+ Lectures 11-12 (inclusive).

## Instructions

+ Type your name and email in the "Student details" section below.
+ Develop the code and generate the figures you need to solve the problems using this notebook.
+ For the answers that require a mathematical proof or derivation you can either:
    
    - Type the answer using the built-in latex capabilities. In this case, simply export the notebook as a pdf and upload it on gradescope; or
    - You can print the notebook (after you are done with all the code), write your answers by hand, scan, turn your response to a single pdf, and upload on gradescope.

+ The total homework points are 100. Please note that the problems are not weighed equally.

**Note**: Please match all the pages corresponding to each of the questions when you submit on gradescope. 

## Student details

+ **First Name:**
+ **Last Name:**
+ **Email:**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('paper')
import numpy as np

## Problem 1

The [San Andreas fault](https://en.wikipedia.org/wiki/San_Andreas_Fault) extends through California forming the boundary between the Pacific and the North American tectonic plates.
It has caused some of the major earthquakes on Earth.
We are going to focus on Southern California and we would like to assess the probability of a major earthquake, defined as an earthquake of magnitude 6.5 or greater, during the next ten years.

A. The first thing we are going to do is go over a [database of past earthquakes](https://scedc.caltech.edu/significant/chron-index.html) that have occured in Southern California and collect the relevant data. We are going to start at 1900 because data before that time may are unreliable.
Go over each decade and count the occurence of a major earthquake (i.e., count the number of organge and red colors in each decade). We have done this for you.

In [ ]:
eq_data = np.array([
    0, # 1900-1909
    1, # 1910-1919
    2, # 1920-1929
    0, # 1930-1939
    3, # 1940-1949
    2, # 1950-1959
    1, # 1960-1969
    2, # 1970-1979
    1, # 1980-1989
    4, # 1990-1999
    0, # 2000-2009
    2 # 2010-2019 
])
fig, ax = plt.subplots(dpi=150)
ax.bar(np.linspace(1900, 2019, eq_data.shape[0]), eq_data, width=10)
ax.set_xlabel('Decade')
ax.set_ylabel('# of major earthquakes in Southern CA');

A. The right way to model the number of earthquakes $X_n$ in a decade $n$ is using a Poisson distribution with unknown rate parameter $\lambda$, i.e.,
$$
X_n | \lambda \sim \operatorname{Poisson}(\lambda).
$$
Here we have $N = 12$ observations, say $x_{1:N} = (x_1,\dots,x_N)$ (stored in ``eq_data`` above).
Find the *joint probability* (otherwise known as the likelihood) $p(x_{1:N}|\lambda)$ of these random variables.<br>
**Answer:**
<br><br><br><br><br><br><br><br>

B. The rate parameter $\lambda$ (number of major earthquakes per ten years) is positive. What prior distribution should we assign to it if we expect it to be around 2?
A convenient choice here is to pick a [Gamma](https://en.wikipedia.org/wiki/Gamma_distribution), see also [the scipy.stats page for the Gamma](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.gamma.html) because it results in an analytical posterior.
We write:
$$
\lambda \sim \operatorname{Gamma}(\alpha, \beta),
$$
where $\alpha$ and $\beta$ are positive *hyper-parameters* that we have to set to represent our prior state of knowledge.
The PDF is:
$$
p(\lambda) = \frac{\beta^\alpha \lambda^{\alpha-1}e^{-\beta \lambda}}{\Gamma(\alpha)},
$$
where we are not conditioning on $\alpha$ and $\beta$ because they should be fixed numbers.
Use the code below to pick some some reasonable values for $\alpha$ and $\beta$.
Hint: Notice that the maximum entropy distribution for a positive parameter with known expectation is the [Exponential](https://en.wikipedia.org/wiki/Exponential_distribution), e.g., see the Table in [this wiki page](https://en.wikipedia.org/wiki/Maximum_entropy_probability_distribution). Then notice that the Exponential is a special case of the Gamma (set $\alpha=1$).

In [ ]:
import scipy.stats as st
alpha = 1.0  # Pick them here
beta = 1.0   # Pick them here
lambda_prior = st.gamma(alpha, scale=1.0 / beta) # Make sure you understand why scale = 1 / beta
lambdas = np.linspace(0, lambda_prior.ppf(0.99), 100)
fig, ax = plt.subplots(dpi=150)
ax.plot(lambdas, lambda_prior.pdf(lambdas))
ax.set_xlabel('$\lambda$ (# or major earthquakes per decade)')
ax.set_ylabel('$p(\lambda)$');

C. Show that the posterior of $\lambda$ conditioned on $x_{1:N}$ is also a Gamma, but with updated hyperparameters.
Hint: When you write down the posterior of $\lambda$ you can drop any multiplicative term that does not depend on it as it will be absorbed in the normalization constnat. This will simplify the notation a little bit.
<br>
**Answer:**
<br><br><br><br><br><br><br><br>

D. Prior-likelihood pairs that result in a posterior with the same form as the prior as known as conjugate distributions. Conjugate distributions are your only hope for analytical Bayesian inference.
As a sanity check, look at the wikipedia page for [conjugate priors](https://en.wikipedia.org/wiki/Conjugate_prior), locate the Poisson-Gamma pair and verify your answer above.

E. Plot the prior and the posterior of $\lambda$ on the same plot.

In [ ]:
alpha_post = 1.0 # Your expression for alpha posterior here
beta_post = 1.0 # Your expression for beta posterior here
lambda_post = st.gamma(alpha_post, scale=1.0 / beta_post)
lambdas = np.linspace(0, lambda_post.ppf(0.99), 100)
fig, ax = plt.subplots(dpi=150)
ax.plot(lambdas, lambda_post.pdf(lambdas))
ax.set_xlabel('$\lambda$ (# or major earthquakes per decade)')
ax.set_ylabel('$p(\lambda|x_{1:N})$');

F. Let's work out the predictive distribution for the number of major earthquakes during the next decade.
This is something that we did not do in class, but it will appear again and again in future lectures.
Let $X$ be the random variable corresponding to the number of major eathquakes during the next decade.
We need to calculate:
$$
p(x|x_{1:N}) = \text{our state of knowledge about $X$ after seeing the data}.
$$
How do we do this?
We just use the sum rule:
$$
p(x|x_{1:N}) = \int_{0}^\infty p(x|\lambda, x_{1:N}) p(\lambda|x_{1:N})d\lambda = \int_{0}^\infty p(x|\lambda) p(\lambda|x_{1:N})d\lambda,
$$
where going from the middle step to the rightmost one we used the assumption that the number of earthquakes occuring in each decade is independent.
Carry out this integral and show that it will give you the [negative Binomial](https://en.wikipedia.org/wiki/Negative_binomial_distribution) distribution $\operatorname{NB}(r,\theta)$, see also the [scipy.stats papge](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.nbinom.html) with parameters
$$
r = \alpha + \sum_{n=1}^N x_n,
$$
and
$$
\theta = \frac{1}{\beta + N + 1}.
$$
**Answer:**
<br><br><br><br><br><br><br><br>

G. Plot the predictive distribution $p(x|x_{1:N})$.

In [ ]:
r = 1.0 # Your expression for r here
theta = 0.2 # Your expression for theta here
X = st.nbinom(r, 1.0 - theta) # Please pay attention to the fact that the wiki and scipy.stats
                              # use slightly different definitions
fig, ax = plt.subplots(dpi=150)
xs = range(0, 10)
ax.vlines(xs, 0, X.pmf(xs), colors='b', lw=5, alpha=0.5)
ax.set_xlabel('$x$ (# of earthquakes during next decade)')
ax.set_ylabel('$p(x)$');

H. What is the probability that at least one major earthquake will occur during the next decade?<br>
**Answer:**
<br><br><br><br><br><br><br><br>

I. What is the probability that at least one major earthquake will occur during the next two decades?<br>
**Answer:**
<br><br><br><br><br><br><br><br>

J. Find a 95\% prediction interval for $\lambda$.

In [ ]:
# Write your code here and print() your answer

K. Find the $\lambda$ that minimizes the absolute loss (see lecture), call it $\lambda^*_N$.
Then, plot the fully Bayesian predictive $p(x|x_{1:N})$ in the same figure as $p(x|\lambda^*_N)$.

In [ ]:
# Write your code here and print() your answer

L. Draw replicated data from the model and compare them to the observed data. Hint: Complete the missing code at the places indicated below.

In [ ]:
# Number of replicated datasets
n_rep = 9
# A variable to store the replicated data:
x_rep = np.ndarray((n_rep, eq_data.shape[0]))
for i in range(n_rep):
    # Student code 1: Take a sample of lambda from its posterior:
    lambda_post_sample = # YOUR CODE HERE
    # Student code 2: Take a sample of size eq_data.shape[0] from the Poisson with parameter
    # lambda_post_sample (You can use st.poisson)
    x_rep[i, :] = # YOUR CODE HERE
fig, ax = plt.subplots(5, 2, sharex='all', sharey='all', figsize=(20, 20))
ax[0, 0].bar(np.linspace(1900, 2019, eq_data.shape[0]), eq_data, width=10, color='red')
for i in range(1, n_rep + 1):
    ax[int(i / 2), i % 2].bar(np.linspace(1900, 2019, eq_data.shape[0]), x_rep[i-1], width=10)

M. Plot the histograms and calculate the Bayesian p-values of the following test-quantities:

+ Maximum number of consecutive decades with no earthquakes.
+ Maximum number of consecutive decades with earthquakes.


In [ ]:
# Define the test quantity as a function of the data:
def T_eq_max_neq(x):
    """
    Return the maximum number of consecutive decades with no earthquakes.
    """
    count = 0
    result = 0
    for i in range(x.shape[0]):
        if x[i] != 0:
            count = 0
        else:
            count += 1
            result = max(result, count)
    return result
    
# The observed test quantity
T_eq_max_neq_obs = T_eq_max_neq(eq_data)
print('The observed test quantity is {0:d}'.format(T_eq_max_neq_obs))
# Draw replicated data
n_rep = 5000
x_rep = np.ndarray((n_rep, eq_data.shape[0]))
for i in range(n_rep):
    # The code below is the same as in P2.L
    # Student code 1: Take a sample of lambda from its posterior:
    lambda_post_sample = # YOUR CODE HERE
    # Student code 2: Take a sample of size eq_data.shape[0] from the Poisson with parameter
    # lambda_post_sample (You can use st.poisson)
    x_rep[i, :] = # YOUR CODE HERE
# Evaluate the test quantity
T_eq_max_neq_rep = np.ndarray(x_rep.shape[0])
for i in range(x_rep.shape[0]):
    T_eq_max_neq_rep[i] = T_eq_max_neq(x_rep[i, :])
# Estimate the Bayesian p-value
p_val = np.sum(np.ones((n_rep,))[T_eq_max_neq_rep > T_eq_max_neq_obs]) / n_rep
print('The Bayesian p_value is {0:1.4f}'.format(p_val))
# Do the plot
fig, ax = plt.subplots(dpi=150)
tmp = ax.hist(T_eq_max_neq_rep, density=True, alpha=0.25, label='Replicated test quantity')[0]
ax.plot(T_eq_max_neq_obs * np.ones((50,)), np.linspace(0, tmp.max(), 50), 'k', label='Observed test quantity')
plt.legend(loc='best');

In [ ]:
# Write your code here for the second test quantity (maximum number of consecutive decades with earthquakes)
# Hint: copy paste your code from the previous cell and make the necessary modifications